# Market depth (order book)


In [1]:
from IPython.display import clear_output
import time
import os
import datetime

os.system("afplay /System/Library/Sounds/Submarine.aiff")
# os.system('echo -n "\a"')  # This will make a beep sound in the terminal

0

In [2]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=10)
# util.logToConsole()

<IB connected to 127.0.0.1:4001 clientId=10>

In [3]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)


def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


# ticker.updateEvent += onTickerUpdate

# IB.sleep(15)

In [4]:
ib.cancelMktDepth(contract)

In [5]:
ib.positions(account="U10394496")

[Position(account='U10394496', contract=Option(conId=637079300, symbol='HYG', lastTradeDateOrContractMonth='20240621', strike=73.0, right='P', multiplier='100', currency='USD', localSymbol='HYG   240621P00073000', tradingClass='HYG'), position=50.0, avgCost=12.44685),
 Position(account='U10394496', contract=Option(conId=682674010, symbol='AAPL', lastTradeDateOrContractMonth='20241018', strike=150.0, right='P', multiplier='100', currency='USD', localSymbol='AAPL  241018P00150000', tradingClass='AAPL'), position=2.0, avgCost=244.69685),
 Position(account='U10394496', contract=Option(conId=669333316, symbol='NVDA', lastTradeDateOrContractMonth='20240719', strike=555.0, right='P', multiplier='100', currency='USD', localSymbol='NVDA  240719P00555000', tradingClass='NVDA'), position=1.0, avgCost=493.0494),
 Position(account='U10394496', contract=Option(conId=613471094, symbol='GOOGL', lastTradeDateOrContractMonth='20240920', strike=120.0, right='P', multiplier='100', currency='USD', localSym

In [28]:
from tools import *

print_all_openorders(ib=ib)

symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1786380182	Submitted	BUY	0.0	1.0		17150.0	
NQ	1786380178	Submitted	BUY	0.0	1.0		17323.0	
NQ	1528433802	Submitted	BUY	0.0	3.0		17526.0	
NQ	1436152242	Submitted	BUY	0.0	1.0		17547.25	
NQ	1939815533	Submitted	SELL	0.0	1.0		17916.0	
NQ	1056387955	PreSubmitted	SELL	0.0	2.0		20480.0	


In [29]:
print_account_summary(ib=ib)

NameError: name 'print_account_summary' is not defined

In [7]:
# default reset values
buy2open = None
sell2close = None

future = [
    pos for pos in ib.positions(account="U10394496") if pos.contract.symbol == "NQ"
][0]

tick_tp = 10
ticrement = 0.25

In [31]:
trades = get_all_openorders(ib, "NQ")
sell2close = [trade for trade in trades if trade.order.permId == 1939815533][0]
print_order(sell2close)

symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1939815533	Submitted	SELL	0.0	1.0		17916.0	


In [35]:
########################################
## BUY TO OPEN SCALP 0.1
########################################

while True:
    clear_output(wait=True)  # Clear the output before printing new output

    # first order of the strategy
    if buy2open is None and sell2close is None:
        os.system("afplay /System/Library/Sounds/Submarine.aiff")
        print("Placing L1.bto")
        action = "BUY"
        totalQuantity = 1
        lmtPrice = ticker.domBids[0].price - tick_tp * ticrement
        print(f"{action}, totalQuantity {totalQuantity}, lmtPrice {lmtPrice}")
        print()

        limit_bto = LimitOrder(
            action=action,
            totalQuantity=totalQuantity,
            lmtPrice=lmtPrice,
            account="U10394496",
        )
        buy2open = ib.placeOrder(contract, limit_bto)
        buy2open_ts = datetime.datetime.now()
        ib.sleep(1)

    print(f"BUY TO OPEN:")
    print_order(buy2open)
    print()

    if buy2open is not None:
        if buy2open.orderStatus.status == "Submitted" and sell2close is None:
            print(
                f"Waiting to get filled L1.bto {buy2open.order.permId} {buy2open.orderStatus.status}"
            )

            if datetime.datetime.now() - buy2open_ts > datetime.timedelta(minutes=5):
                print("Cancelling order due to timeout")
                ib.cancelOrder(buy2open.order)
                ib.sleep(1)
                print(buy2open.log)
            print()
        if buy2open.orderStatus.status == "Filled" and sell2close is None:
            os.system("afplay /System/Library/Sounds/Submarine.aiff")
            print("Placing L2.stc")
            action = "SELL"
            totalQuantity = 1
            lmtPrice = buy2open.orderStatus.avgFillPrice + tick_tp * ticrement
            print(f"{action}, totalQuantity {totalQuantity}, lmtPrice {lmtPrice}")
            limit_stc = LimitOrder(
                action=action,
                totalQuantity=totalQuantity,
                lmtPrice=lmtPrice,
                account="U10394496",
            )
            sell2close = ib.placeOrder(contract, limit_stc)
            ib.sleep(1)
        elif (
            buy2open.orderStatus.status == "Inactive"
            or buy2open.orderStatus.status == "Cancelled"
        ) and sell2close is None:
            print("***** order is inactive *****")
            print(buy2open.log)
            print("*****************************")
            buy2open = None

    print(f"SELL TO CLOSE:")
    print_order(sell2close)

    if sell2close is not None:
        if sell2close.orderStatus.status == "Filled":
            os.system("afplay /System/Library/Sounds/Submarine.aiff")
            print("L2.stc filled @ {}".format(sell2close.orderStatus.avgFillPrice))
            buy2open = None
            sell2close = None
    print()
    print()
    print("ALL OPEN ORDERS:")
    print_all_openorders(ib=ib)
    print()

    # get current position of contract NQM2024
    print("Current position:")
    print(
        f"{future.contract.symbol}:  {future.position} @ {future.avgCost/float(contract.multiplier)}"
    )

    # os.system("afplay /System/Library/Sounds/Submarine.aiff")

    time.sleep(1)
    print(".")
    time.sleep(1)
    print(".")
    time.sleep(1)
    print(".")
    time.sleep(1)
    print(".")
    time.sleep(1)
    print(".")
    time.sleep(1)

BUY TO OPEN:
None

SELL TO CLOSE:
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1939815533	Submitted	SELL	0.0	1.0		17916.0	


ALL OPEN ORDERS:
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1786380182	Submitted	BUY	0.0	1.0		17150.0	
NQ	1786380178	Submitted	BUY	0.0	1.0		17323.0	
NQ	1528433802	Submitted	BUY	0.0	3.0		17526.0	
NQ	1436152242	Submitted	BUY	0.0	1.0		17547.25	
NQ	1939815533	Submitted	SELL	0.0	1.0		17916.0	
NQ	1056387955	PreSubmitted	SELL	0.0	2.0		20480.0	

Current position:
NQ:  -6.0 @ 17905.0535435925
.
.
.


In [14]:
# get a specific order by id
trades = get_all_openorders()
sell2close = [trade for trade in trades if trade.order.permId == 1939815533]
print(sell2close[0].orderStatus)

OrderStatus(orderId=40, status='Submitted', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=1939815533, parentId=0, lastFillPrice=0.0, clientId=10, whyHeld='', mktCapPrice=0.0)


In [20]:
acct_fields = ib.accountSummary(account="U10394496")

for f in acct_fields:
    if "DayTrades" not in f.tag:
        print(f.tag, ":", f.value)

AccountType : INDIVIDUAL
Cushion : 0.270221
LookAheadNextChange : 1714483800
AccruedCash : 0.00
AvailableFunds : 38356.40
BuyingPower : 208850.69
EquityWithLoanValue : 52212.67
ExcessLiquidity : 52212.67
FullAvailableFunds : 38356.40
FullExcessLiquidity : 52212.67
FullInitMarginReq : 152418.94
FullMaintMarginReq : 138562.67
GrossPositionValue : 2447.07
InitMarginReq : 152418.94
LookAheadAvailableFunds : 49644.45
LookAheadExcessLiquidity : 93781.47
LookAheadInitMarginReq : 106693.26
LookAheadMaintMarginReq : 96993.87
MaintMarginReq : 138562.67
NetLiquidation : 193222.41
PreviousDayEquityWithLoanValue : 0.00
RegTEquity : 0.01
RegTMargin : 0.00
SMA : 623470.03
TotalCashValue : 190775.34


In [15]:
from ib_async import OrderState

# Get all accounts
accounts = ["U14726869"]

# Create a dictionary to store the portfolio summary for each account
portfolios = {}

# Iterate over each account
for account in accounts:
    # Get the positions and orders for the account
    positions = ib.positions(account)
    orders = ib.orders()

    # Calculate the total value and number of positions
    total_value = 0
    num_positions = 0
    for position in positions:
        total_value += position.marketValue()
        num_positions += 1

    # Calculate the total value of open orders
    open_order_value = 0
    for order in orders:
        if order.orderStatus.status == OrderState.LIVE:
            open_order_value += order.totalValue()

    # Store the portfolio summary in the dictionary
    portfolios[account.account] = {
        "Account": account.account,
        "Total Value": total_value,
        "Number of Positions": num_positions,
        "Open Order Value": open_order_value,
    }

# Convert the dictionary to a Pandas DataFrame
portfolios_df = pd.DataFrame(list(portfolios.values()))

# Print the portfolio summary in a nice table form
print(portfolios_df.to_string(index=False))

AttributeError: 'Order' object has no attribute 'orderStatus'

## REVERSE STRATEGY - DRAFT - DO NOT RUN


In [196]:
# L1 = buy @ mid - 10 tickets , tp + 10 ticks

# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)


qty_target = -7

# get current position of contract nq24
position = ib.positions(account="U10394496")
future = position[0]

openorders = get_all_openorders(sym="NQ")


if future.position == qty_target:
    print("Position is at target quantity / {}, {}".format(future.position, qty_target))
    # check for initiation order (L1.buy)
    if opentobuy is None:
        print("Initiation Order does not exist")
        print()

        print("Placing L1.buy")
        action = "BUY"
        totalQuantity = 1
        opentobuy_lmtPrice = ticker.domBids[0].price
        lmtPrice = opentobuy_lmtPrice
        print("Setting open to buy order:")
        print(f"{action}, totalQuantity {totalQuantity}, lmtPrice {lmtPrice}")
        print("Setting takeprofit -> None")
        print()

        limitbuy = LimitOrder(
            action=action,
            totalQuantity=totalQuantity,
            lmtPrice=lmtPrice,
            account="U10394496",
        )
        # opentobuy = ib.placeOrder(contract, limitbuy)
        # ib.sleep(1)
    else:
        print("Initiation Order exists...")

    print(opentobuy)
    print()

if future.position > qty_target:
    print("Future.position {} > {} qty_target".format(future.position, qty_target))
    print()
    # check for take profit order (L1.sell)
    if takeprofit is None:
        print("TP Order does not exist")
        print("Placing TP Order")
        print()

        # place L1.sell
        action = "SELL"
        totalQuantity = 1
        lmtPrice = opentobuy.orderStatus.avgFillPrice + 10 * 0.25
        print(f"{action}, totalQuantity {totalQuantity}, lmtPrice {lmtPrice}")
        limitsell = LimitOrder(
            action=action,
            totalQuantity=totalQuantity,
            lmtPrice=lmtPrice,
            account="U10394496",
        )

        # takeprofit = ib.placeOrder(contract, limitsell)
        # ib.sleep(1)
    else:
        print("Take Profit Order exists...")
        print()

    print(takeprofit)
    print()

nq_position = [
    pos for pos in ib.positions(account="U10394496") if pos.contract.symbol == "NQ"
][0]
print(
    f"Symbol {nq_position.contract.symbol} Position {nq_position.position} avgCost {nq_position.avgCost/20}"
)
print()
futures_port = [p for p in ib.portfolio() if p.contract.symbol == "NQ"][0]
print(
    f"RealizedPNL {futures_port.realizedPNL} UnrealizedPNL {futures_port.unrealizedPNL} MarketValue {futures_port.marketValue/1e6}mm"
)

Open to Buy Order:
TradeLogEntry(time=datetime.datetime(2024, 4, 29, 8, 30, 40, 406788, tzinfo=datetime.timezone.utc), status='Filled', message='', errorCode=0)

Take Profit Order:
LimitOrder(action='SELL', totalQuantity=1, lmtPrice=17905.75, account='U10394496')

Ask Price 17895.25
Bid Price 17894.5

Future.position -6.0 > -7 qty_target

Take Profit Order exists...

LimitOrder(action='SELL', totalQuantity=1, lmtPrice=17905.75, account='U10394496')

Symbol NQ Position -6.0 avgCost 17892.076428275

RealizedPNL -3175.17 UnrealizedPNL -110.83 MarketValue -2.14716mm


In [ ]:
# canceled orders
print(f"permId\t\taction\t\ttotalQuantity\tfilledQuantity\t\tlmtPrice")

closedOrders = ib.orders()
[
    print(
        f"{order.permId}\t{order.action}\t\t{order.totalQuantity}\t\t{order.filledQuantity}\t\t\t{order.lmtPrice}"
    )
    for order in closedOrders
    if order.filledQuantity == 0
]

In [ ]:
#########################################
### SELL TO OPEN SCALP 0.1
#########################################

print("STO: ", sell2open)
print()

tick_tp = 10
ticrement = 0.25

# first order of the strategy
if sell2open is None:
    print("Placing L1.STO")
    action = "SELL"
    totalQuantity = 1
    lmtPrice = ticker.domAsks[0].price + tick_tp * ticrement
    print(f"{action}, totalQuantity {totalQuantity}, lmtPrice {lmtPrice}")
    print()

    limit_sto = LimitOrder(
        action=action,
        totalQuantity=totalQuantity,
        lmtPrice=lmtPrice,
        account="U10394496",
    )
    sell2open = ib.placeOrder(contract, limit_sto)
    ib.sleep(1)
else:
    print(f"L1.STO {sell2open.order.permId} {sell2open.orderStatus.status}")
    print()

    if sell2open.orderStatus.status == "Filled":
        print("L1.STO filled @ {}".format(sell2open.orderStatus.avgFillPrice))

print()

if sell2open is not None:
    if sell2open.orderStatus.status == "Submitted" and buy2close is None:
        print(
            f"Waiting to get filled L1.STO {sell2open.order.permId} {sell2open.orderStatus.status}"
        )
        print()
    if sell2open.orderStatus.status == "Filled" and buy2close is None:
        os.system("afplay /System/Library/Sounds/Submarine.aiff")

        print("Placing L2.BTC")
        action = "BUY"
        totalQuantity = 1
        lmtPrice = sell2open.orderStatus.avgFillPrice + tick_tp * ticrement
        print(f"{action}, totalQuantity {totalQuantity}, lmtPrice {lmtPrice}")
        limit_btc = LimitOrder(
            action=action,
            totalQuantity=totalQuantity,
            lmtPrice=lmtPrice,
            account="U10394496",
        )
        # buy2close = ib.placeOrder(contract, limit_btc)
        # ib.sleep(1)
    elif sell2open.orderStatus.status == "Inactive" and buy2close is None:
        print(f"L1.STO {sell2open.order.permId} {sell2open.OrderStatus.status}")
    elif buy2close is not None:
        print("L2.BTC {buy2close.order.permId} {buy2close.OrderStatus.status}")
        print()

        if buy2close.orderStatus.status == "Filled":
            print(
                "L2.BTC already filled @ {}".format(sell2open.orderStatus.avgFillPrice)
            )
else:
    print("N/A Condition")

STO:  Trade(contract=Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), order=LimitOrder(orderId=21, clientId=10, permId=1939815514, action='SELL', totalQuantity=1.0, lmtPrice=17911.75, auxPrice=0.0, account='U10394496'), orderStatus=OrderStatus(orderId=21, status='Inactive', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=1939815514, parentId=0, lastFillPrice=0.0, clientId=10, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 4, 29, 23, 7, 46, 638519, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 4, 29, 23, 7, 46, 896837, tzinfo=datetime.timezone.utc), status='Inactive', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 4, 29, 23, 7, 46, 913501, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 201, re

AttributeError: 'Trade' object has no attribute 'OrderStatus'